# Mapping Molecules

In this notebook we will cover how to map molecules in different ways and look at some of the things we can do with them.

Throughout this demonstration we will load data from a GROMACS simulation and therefore, we need to define a set of units and a file reader object to use. For this reason, we have changed the imports a little bit to keep the code to minimum.

In [1]:
import mdsuite as mds
import mdsuite.file_io.chemfiles_read
from mdsuite.utils import Units

from zinchub import DataHub
import shutil

import h5py as hf
import numpy as np

### Loading the data

In this tutorial we are using 50 ns simulations of 14 water molecules in a continuum fluid performed with GROMACS. We will use pure atomistic naming as well as ligand naming, the topology files for which are contained on DataHub.

In [2]:
water = DataHub(url="https://github.com/zincware/DataHub/tree/main/Water_14_Gromacs")
water.get_file('./')
file_paths = [
        f for f in water.file_raw
    ]

### Project definition

Here we create the project and define some custom units used by GROMACS.

In [3]:
project = mds.Project("Mapping_Molecules")

gmx_units = Units(
        time=1e-15,
        length=1e-10,
        energy=1.6022e-19,
        NkTV2p=1.6021765e6,
        boltzmann=8.617343e-5,
        temperature=1,
        pressure=100000,
    )

2022-01-26 14:48:53,358 - INFO: Creating new project Mapping_Molecules


### Mapping molecules with SMILES

In this section we take a look at how one can map molecules using SMILES strings.

In [4]:
traj_path = file_paths[2]
topol_path = file_paths[0]

file_reader = mdsuite.file_io.chemfiles_read.ChemfilesRead(
    traj_file_path=traj_path, topol_file_path=topol_path
)

water_chemical = project.add_experiment(
    name=f"water_chemical",
    timestep=0.002,
    temperature=300.0,
    units=gmx_units,
    simulation_data=file_reader,
)
water_chemical.run.CoordinateUnwrapper()

2022-01-26 14:48:53,425 - INFO: Creating a new experiment!


100%|███████████████████████████████████| 1/1 [00:00<00:00,  2.74it/s]
Applying transformation 'Unwrapped_Positions' to 'O': 100%|█| 1/1 [00:
Applying transformation 'Unwrapped_Positions' to 'H': 100%|█| 1/1 [00:


In [5]:
water_chemical.run.MolecularMap(
    molecules={"water": {"smiles": "[H]O[H]", "amount": 14, "cutoff": 1.7}}
)

2022-01-26 14:48:54,175 - INFO: Building molecular graph from configuration for water


100%|████████████████████████████████| 42/42 [00:00<00:00, 689.35it/s]


2022-01-26 14:48:54,356 - INFO: Performing molecule number isomorphism test.
2022-01-26 14:48:54,356 - INFO: Amount of molecules test passed.
2022-01-26 14:48:54,357 - INFO: Performing group equality isomorphism test.
2022-01-26 14:48:54,357 - INFO: Group equality isomorphism test passed.
2022-01-26 14:48:54,400 - INFO: Mapping molecule graphs onto trajectory for water


2it [00:00, 27.88it/s]                                                
Applying transformation 'Unwrapped_Positions' to 'water': 100%|█| 1/1 


In [6]:
water_chemical.molecules

{'water': {'n_particles': 14,
  'indices': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
  'mass': 18.015,
  'groups': {'0': {'H': [0, 1], 'O': [0]},
   '1': {'H': [2, 3], 'O': [1]},
   '2': {'H': [4, 5], 'O': [2]},
   '3': {'H': [6, 7], 'O': [3]},
   '4': {'H': [8, 9], 'O': [4]},
   '5': {'H': [10, 11], 'O': [5]},
   '6': {'H': [12, 13], 'O': [6]},
   '7': {'H': [14, 15], 'O': [7]},
   '8': {'H': [16, 17], 'O': [8]},
   '9': {'H': [18, 19], 'O': [9]},
   '10': {'H': [20, 21], 'O': [10]},
   '11': {'H': [22, 23], 'O': [11]},
   '12': {'H': [24, 25], 'O': [12]},
   '13': {'H': [26, 27], 'O': [13]}}}}

### Mapping Molecules with a reference dict

If you do not have particles with chemical names but you nonetheless wish to construct groups out of particles, this can be achieved by using a reference dict.

In this example, we use the ligand naming from GROMACS to construct water molecules.

In [7]:
traj_path = file_paths[2]
topol_path = file_paths[1]

file_reader = mdsuite.file_io.chemfiles_read.ChemfilesRead(
    traj_file_path=traj_path, topol_file_path=topol_path
)

water_ligand = project.add_experiment(
    name=f"water_ligand",
    timestep=0.002,
    temperature=300.0,
    units=gmx_units,
    simulation_data=file_reader,
)

2022-01-26 14:48:54,662 - INFO: Creating a new experiment!
2022-01-26 14:48:55,344 - WARNING: WARNING element OW has been assigned mass=0.0


100%|███████████████████████████████████| 1/1 [00:00<00:00,  2.42it/s]


Keep in mind, as the particles are not named from the periodic tables, important properties such as mass will need to be filled in manually.

In [8]:
water_ligand.species['OW'].mass = [15.999]
water_ligand.species['HW1'].mass = [1.00784]
water_ligand.species['HW2'].mass = [1.00784]

In [9]:
water_ligand.run.CoordinateUnwrapper()

water_ligand.run.MolecularMap(
    molecules={"water": {"reference": {"HW1": 1, "OW": 1, "HW2": 1}, "amount": 14, "cutoff": 1.7}}
)

Applying transformation 'Unwrapped_Positions' to 'OW': 100%|█| 1/1 [00
Applying transformation 'Unwrapped_Positions' to 'HW1': 100%|█| 1/1 [0
Applying transformation 'Unwrapped_Positions' to 'HW2': 100%|█| 1/1 [0

2022-01-26 14:48:57,723 - INFO: Building molecular graph from configuration for water



100%|████████████████████████████████| 42/42 [00:00<00:00, 693.51it/s]


2022-01-26 14:48:57,899 - INFO: Performing molecule number isomorphism test.
2022-01-26 14:48:57,900 - INFO: Amount of molecules test passed.
2022-01-26 14:48:57,900 - INFO: Performing group equality isomorphism test.
2022-01-26 14:48:57,901 - INFO: Group equality isomorphism test passed.
2022-01-26 14:48:57,944 - INFO: Mapping molecule graphs onto trajectory for water


2it [00:00, 31.26it/s]                                                
Applying transformation 'Unwrapped_Positions' to 'water': 100%|█| 1/1 


In [10]:
water_ligand.molecules

{'water': {'n_particles': 14,
  'indices': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
  'mass': 18.014680000000002,
  'groups': {'0': {'HW1': [0], 'OW': [0], 'HW2': [0]},
   '1': {'HW1': [1], 'OW': [1], 'HW2': [1]},
   '2': {'HW1': [2], 'OW': [2], 'HW2': [2]},
   '3': {'HW1': [3], 'OW': [3], 'HW2': [3]},
   '4': {'HW1': [4], 'OW': [4], 'HW2': [4]},
   '5': {'HW1': [5], 'OW': [5], 'HW2': [5]},
   '6': {'HW1': [6], 'OW': [6], 'HW2': [6]},
   '7': {'HW1': [7], 'OW': [7], 'HW2': [7]},
   '8': {'HW1': [8], 'OW': [8], 'HW2': [8]},
   '9': {'HW1': [9], 'OW': [9], 'HW2': [9]},
   '10': {'HW1': [10], 'OW': [10], 'HW2': [10]},
   '11': {'HW1': [11], 'OW': [11], 'HW2': [11]},
   '12': {'HW1': [12], 'OW': [12], 'HW2': [12]},
   '13': {'HW1': [13], 'OW': [13], 'HW2': [13]}}}}

### What information is stored?

So the molecule mapping itself was quick and easy, but what information has been stored along the way?

All meta-data about the molecules is stored in the experiment class under molecules. Let's take a look at what this contains.

In [11]:
water_chemical.molecules.keys()

dict_keys(['water'])

This dict will contain all of the molecules that have been mapped, but this is not the information about the molecules, for that, we need to look at the water molecule.

In [12]:
water_chemical.molecules['water'].keys()

dict_keys(['n_particles', 'indices', 'mass', 'groups'])

Three of these are fairly trivial and we can look at them quickly, groups will require some more attention.

In [13]:
print(f"n_particles: {water_chemical.molecules['water']['n_particles']}")
print(f"indices: {water_chemical.molecules['water']['indices']}")
print(f"mass: {water_chemical.molecules['water']['mass']}")

n_particles: 14
indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
mass: 18.015


Now let's take a look at the groups key.

In [14]:
print(water_chemical.molecules['water']['groups'])

{'0': {'H': [0, 1], 'O': [0]}, '1': {'H': [2, 3], 'O': [1]}, '2': {'H': [4, 5], 'O': [2]}, '3': {'H': [6, 7], 'O': [3]}, '4': {'H': [8, 9], 'O': [4]}, '5': {'H': [10, 11], 'O': [5]}, '6': {'H': [12, 13], 'O': [6]}, '7': {'H': [14, 15], 'O': [7]}, '8': {'H': [16, 17], 'O': [8]}, '9': {'H': [18, 19], 'O': [9]}, '10': {'H': [20, 21], 'O': [10]}, '11': {'H': [22, 23], 'O': [11]}, '12': {'H': [24, 25], 'O': [12]}, '13': {'H': [26, 27], 'O': [13]}}


The groups key contains direct information about which atoms belong to which molecule, for example, the 10th molecule of water (id=9) consists of Hydrogen atoms 18 and 19 and oxygen atom 10.

In [15]:
print(water_chemical.molecules['water']['groups']['9'])

{'H': [18, 19], 'O': [9]}


With this information you can compute values, e.g. diffusion coefficients with only the atoms belonging to a single molecule using the atom_select arguments in the calculator.

### Analysis with molecules

Now that we have seen how we can build molecules and what information this gives is, let's look at what we can analyse using them.

#### Angular Distribution Functions (ADFS)

First things first, let's confirm we are working with water by looking at the angular distribution function of the atoms.

In [ ]:
water_chemical.run.AngularDistributionFunction(
    number_of_configurations=4000, number_of_bins=500, norm_power=8
)

  0%|                                           | 0/1 [00:00<?, ?it/s]

Looking at the O_H_H ADF in he top right we see a strong max peak at 109.591 degrees corresponding well with the bond angle of an SPCE model (109.47) as was used in the simulation. It is also worth noting that the oxygen triplet angle looks similar to that measured in QM and experimental studies.

When we want to study the molecular ADF we have two choices, we can either pass it as a species argument to the calculator if only one is desired, we we can call the calculator with the `molecules=True` keyword as we will do here.

In [ ]:
water_chemical.run.AngularDistributionFunction(
    molecules=True, number_of_configurations=3000, number_of_bins=500, norm_power=8
)

In this case we have increased the norm power to suppress the noise floor and highlight only the most dominant peaks.

In the water molecule ADF it we do not see any clear stacking or structure suggesting there is not special organization of the molecules in these simulations.

#### Radial Distribution Functions (RDFs)

Now let's look at the radial structure and distribution of particles in space of both the atomistic system and the molecules. This is where molecule mapping can be very helpful as often we are more interested in the positions of the molecules themselves and not necessarily those of the atoms.

In [ ]:
water_chemical.run.RadialDistributionFunction(
    number_of_configurations=4000, start=100, stop=5100, number_of_bins=500
)

In the case of the hydrogen-hydrogen and the oxygen-hydrogen we can see clear peaks where the bond distance is fixed. Using the cursor to hover over the points in the plot we can identify a bond distance between hydrogens of approximately 0.163 nm, in good agreement with experimental values. The oxygen-hydrogen bond sits around 0.09 nm, also in good agreement with experiment values.

In [ ]:
water_ligand.run.RadialDistributionFunction(
    number_of_configurations=5200, start=0, stop=5100, number_of_bins=500, molecules=True
)

### Diffusion Coeffients

Finally, let's start looking at the diffusion coefficients of the atoms and molecules.

In [ ]:
water_chemical.run.EinsteinDiffusionCoefficients(data_range=500)

In [ ]:
water_chemical.run.EinsteinDiffusionCoefficients(molecules=True, data_range=500)

### Group-wise analysis

Say we want to study a specific molecule. We only want the diffusion coefficients, ADFs, and RDFs of the atoms in that one molecule. This can be achieved with the MDSuite atom-selection command and is included here as a demonstration of the flexibility of the software.

First things first, let's select a molecule group to study, say the first water molecule.

In [ ]:
water_group = water_chemical.molecules['water']['groups']['0']
print(water_group)

In [ ]:
water_chemical.run.RadialDistributionFunction(atom_selection={'H': [0, 1], 'O': [0]}, number_of_configurations=2517)

In [ ]:
water_chemical.run.AngularDistributionFunction(atom_selection={'H': [0, 1], 'O': [0]}, number_of_configurations=100)

In [ ]:
water_chemical.run.EinsteinDiffusionCoefficients(atom_selection=water_group, data_range=2702)

In [ ]:
project.run.KirkwoodBuffIntegral()

### Some notes on mapping molecules

* Make sure the reference configuration you use to map the molecules is equilibrated. If not, atoms may be too close together and you will find it difficult to map the molecules correctly and pass all of the isomorphism tests.
* If too few, reduce the cutoff, if too many, increase it.
* If you find that you need an extremely fine cutoff to get the correct number of molecules, it is likely that the atoms are very close together and you will pick up other groups leading to the next isomorphism test to fail. Try using a different reference configuration.